## 保存数据到mysql数据库，包含全部个股数据

### 基础引用初始化

In [2]:
#常规引用
import time
import tushare as ts
import pandas as pd
import numpy as np
import baostock as bs

today=time.strftime('%Y-%m-%d',time.localtime(time.time()))


In [10]:
# from sqlalchemy import create_engine

# #获取股票代码清
# engine = create_engine("mysql+pymysql://{}:{}@{}/{}?charset={}".format('root', 'qqq375701956', '127.0.0.1:3306', 'finance','utf8'))
# conn = engine.connect()#创建连接


### baostock接口调用函数
适用于多天数据为更新，所花时间较长



#### 获取股票列表

In [11]:

stocks=ts.get_stock_basics().sort_index()
stocks.head(2)

,name,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
000001,平安银行,银行,深圳,9.61,194.06,194.06,37076.83,0.00,105.87,808.16,...,13.82,1.13,19910403,1152.93,5.94,18.80,15.47,0.00,22.94,299958.0
000002,万 科Ａ,全国地产,深圳,12.55,97.15,113.02,16387.63,13581.85,124.12,119.31,...,14.80,1.82,19910129,981.54,8.68,27.21,30.43,35.99,8.15,290546.0


In [12]:
print(stocks.shape)

stocks=stocks[stocks.index.str[:3]!='688']  # 剔除科创板股票(688开头)
print(stocks.shape)
# stocks.index.str[0:1]

(3747, 22)
(3683, 22)


#### 调用接口函数

In [22]:
#调用接口函数
def addcode(c): #在代码后载入sz，sh
    return c+'.sh'  if c[0:1]=='6' else c+'.sz' 
def precode(c): #在代码后载入sz，sh
    return 'sh.'+c  if c[0:1]=='6' else 'sz.'+c 

def getBaostock(pcode,start_date,end_date,datatype):
#     print(code)
    rs = bs.query_history_k_data_plus(pcode,
    "date,code,close,open,low,high,preclose,volume,amount,turn,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST",  #preclose,turn,
    start_date=start_date, end_date=end_date,
    frequency=datatype, adjustflag="3")#复权类型选择123
    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    df=pd.DataFrame(data_list, columns=rs.fields)
    df.close=pd.to_numeric(pd.to_numeric(df.close, downcast='float')*100,downcast='integer')  # 收盘价*100转整型数
    df.open=pd.to_numeric(pd.to_numeric(df.open, downcast='float')*100,downcast='integer')
    df.low=pd.to_numeric(pd.to_numeric(df.low, downcast='float')*100,downcast='integer')
    df.high=pd.to_numeric(pd.to_numeric(df.high, downcast='float')*100,downcast='integer')
    df.preclose=pd.to_numeric(pd.to_numeric(df.preclose, downcast='float')*100,downcast='integer')
    df.turn=pd.to_numeric(df.turn, downcast='float')
    df['flowmkt']=0.001*df.close*df.volume/df.turn
    df['pctChg']=100*df.close/df.preclose-100
    df['ep']=100/pd.to_numeric(df.peTTM, downcast='float')
    df['eb']=100/pd.to_numeric(df.pbMRQ, downcast='float')
    df.amount=pd.to_numeric(df.amount, downcast='float')/10000
    df.volume=pd.to_numeric(df.volume,downcast='float')/10000
    df.fillna(method = 'ffill',inplace=True)
    df.dropna(inplace=True)
    return df

#### 依次添加baostock获取的数据到mysql


In [23]:
# def getStocksData(codes,start_date,issql=0):
#     dfdata=pd.DataFrame()
#     for code in codes:
# #         try:
#         df=getBaostock(precode(code),start_date,today,'d')
#         if issql==1:
#             if df.shape[0]>0 :
#                 df['code']=code
#                 df.to_sql(name='k'+code[0:4], con=conn, if_exists='append', index=False)

#             else:
#                 print(code+' empty data')
#         else:
#             df=pd.DataFrame()
#             dfdata.append(df)
# #         except:
# #             print(code,'error')
            
# import threading

# start_time=time.time()
# threads=[]
# codes=stocks.index[:]
# m=20
# start_date='2010-01-01'
# for i in range(1):
#     lg = bs.login()

#     threads.append(threading.Thread(target=getStocksData,args=(codes[i*m:i*m+m],start_date,1,)))
#     threads[i].start()
#     bs.logout()

# thread2.join()

# for i in range(4):
#     threads[i].join()
#     getStocksData(codes,issql=0)

In [24]:
### 登陆系统 ####
lg = bs.login()
# 显示登陆返回信息
start_date='2010-01-01'
print('login respond  msg:'+lg.error_msg)
i=0
symbols=stocks.index[:2]
# symbols=['']
for code in symbols:
        i+=1
#     if i%100==0:
#         print(i,'of',len(stocks),code)
#     try:
        df=getBaostock(precode(code),start_date,today,'d')
        print(df)
        df.code=code
        print(df)

        if df.shape[0]>0:
            df.to_sql(name='k'+code[0:4], con=conn, if_exists='append', index=False)
    #             df.to_csv(code+'.csv')
    #         mcodes.append(code)
        else:
            print(code+' empty data')
#     except:
#         print(code,'error')
bs.logout()
df.head()

login success!
login respond  msg:success
0       2371.000000
1       2330.000000
2       2290.000000
3       2265.000000
4       2260.000000
           ...     
2399    1645.000122
2400    1641.000000
2401    1585.000000
2402    1586.000000
2403    1559.000000
Name: close, Length: 2404, dtype: float32


TypeError: can't multiply sequence by non-int of type 'float'

### 获取tushare当日全部数据到mysql
适用于当日数据更新，所花时间较短

In [ ]:
kdata=ts.get_today_all()
jdata=kdata[['code','trade','open','settlement','low','high','turnoverratio','volume','amount']]
jdata.columns=['code','close','open','preclose','low','high','turn','volume','amount']  # 列名不一样，重新设置一下
jdata['date']=today
# jdata.to_csv(today+'.csv')


In [ ]:
# jdata=kdata[['code','trade','open','settlement','low','high','turnoverratio','volume','amount']]
# jdata.columns=['code','close','open','preclose','low','high','turn','volume','amount']
# jdata['date']=today
# jdata['date']='2019-08-09'
jdata.head()


In [ ]:
for i in range(jdata.shape[0]):
    if i%500==1:
        print(i , 'of',jdata.shape[0],jdata.code[i])
#     print(jdata.code[i])
    jdata.iloc[i:i+1].to_sql(name='k'+jdata.code[i][0:4], con=conn, if_exists='append', index=False)

## 保存各类指数数据到数据库

In [ ]:
##调用接口函数
### 登陆系统 ####
def get_index(codes,start_date):
    lg = bs.login()
    for code in codes:
        # 显示登陆返回信息
        df=getBaostock(code,start_date,today)
        df.code=code

        if df.shape[0]>0:
            df.to_sql(name='k'+code[:4].replace('.',''), con=conn, if_exists='append', index=False)
        else:
            print(code+' empty data')
    bs.logout()
    print('save to mysql ok')



In [ ]:
# aindxname=pd.read_csv(u'/Users/hongyuouyang/python/finance/indexname.csv')
# aindxname.to_sql(name='indexname', con=conn, if_exists='append', index=False)


In [ ]:
start_date='2019-01-01'

aindxname=pd.read_sql('select * from indexname',con=conn)
get_index(aindxname.code.values,start_date)